# Republic frase modellen

Doel:

- informatie extractie uit de resoluties, gebruik makend van de repetitieve aspecten van de teksten, formulaische uitdrukkingen, signaalwoorden en onderwerpstermen.
- het opbouwen van collectie-specifieke frase-modellen (lijstjes met termen en frasen, varianten daarvan en metadata categorieën en labels) zodat we informatie kunnen extraheren en om kunnen vormen tot inhoudelijke metadata voor ontsluiting van de individuele resoluties. 

Aanpak: harvesting phrases o.b.v. frequentielijsten

Overlap/verschil tussen jaren:

- tussen twee opvolgende jaren
- tussen twee jaren met een decennium ertussen
- tussen twee jaren met X jaren ertussen

Overlap tussen resoluties en respectenlijsten:

- in hoeverre komen termen uit de respecten letterlijk voor in de resoluties?
- vergelijk e.g. automatische extractie en respecten. Dit vergt ook interactie met Team Text en hun NER activiteiten

## Overlap analyseren

**doel:** bepalen welke termen structureel vaak voorkomen en hoe ze kunnen worden gebruikt om de inhoud van resoluties te analyseren

Lijstjes:

- hoedanigheden, rollen (ambassadeur, graaf, suppliant)
    - dit kan gebruikt worden om o.a. persoonsnamen te spotten (dus nuttig voor NER trainen of resultaten opschonen)
    - signaalwoorden/-frasen:
        - <ambassadeur> in, uit, van, bij <land/hof>
        - uit, naar, tot, te <plaats>
        - gecommiteerd/commissie
- documenten:
    - Placaat
    - Rapport
    - Paspoort
- onderwerpen:
    - financiën
    - rechten


In [65]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_dir = os.path.split(os.getcwd())[0]
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

In [69]:
# Laad de benodigde modules in
import re
from collections import Counter
from collections import defaultdict
from typing import Union
from republic.model.inventory_mapping import get_inventory_by_num
from republic.config.republic_config import base_config, set_config_inventory_num
import republic.elastic.republic_elasticsearch as rep_es

# Een handige 'Key Word In Context' functie om keywords/frasen en omringende tekst te zien
def get_keyword_context(text: str, keyword: str, 
                        context_size: int = 3, 
                        prefix_size: Union[None, int] = None, 
                        suffix_size: Union[None, int] = None):
    if not prefix_size:
        prefix_size = context_size
    if not suffix_size:
        suffix_size = context_size
    text = re.sub(r'\s+',' ', text)
    prefix_pattern = r'(\w*\W*){' + f'{prefix_size}' + '}$'
    suffix_pattern = r'^(\w*\W*){' + f'{suffix_size}' + '}'
    contexts = []
    for match in re.finditer(r'\b' + keyword + r'\b', text):
        prefix_window = text[:match.start()]
        suffix_window = text[match.end():]
        prefix_terms = prefix_window.strip().split(' ')
        suffix_terms = suffix_window.strip().split(' ')
        prefix = ' '.join(prefix_terms[-prefix_size:])# re.search(prefix_pattern, prefix_window)
        suffix = ' '.join(suffix_terms[:suffix_size])# re.search(suffix_pattern, suffix_window)
        context = {
            'keyword': keyword,
            'prefix': prefix,
            'suffix': suffix,
            'keyword_offset': match.start(),
            'prefix_offset': match.start() - len(prefix),
            'suffix_offset': match.end()
        }
        contexts.append(context)
    return contexts


es_republic = rep_es.initialize_es(host_type='external', timeout=60)



In [71]:
# kies een inventaris nummer om te doorzoeken
inv_num = 3761
# de base_dir is niet relevant maar wel nodig voor het maken van een config object
base_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/"
# genereer een config object voor het specifieke inventaris nummer dat je wilt doorzoeken
inv_config = set_config_inventory_num(base_config, inv_num, base_dir, ocr_type='pagexml')

# haal alle paragraphs van het inventaris op uit de index
#docs = rep_es.retrieve_paragraphs_by_inventory(es_republic, inv_num, inv_config)

print('number of paragraphs:', len(docs))

# hoe ziet een paragraph document eruit?
print(json.dumps(docs[0], indent=4))

number of paragraphs: 5729
{
    "metadata": {
        "id": "NL-HaNA_3761_0005-page-9-col-0-tr-0",
        "inventory_num": 3761,
        "scan_num": 5,
        "scan_id": "NL-HaNA_3761_0005",
        "page_num": 9,
        "page_id": "NL-HaNA_3761_0005-page-9",
        "column_index": 0,
        "column_id": "NL-HaNA_3761_0005-page-9-col-0",
        "scan_type": "pagexml",
        "page_type": "resolution_page"
    },
    "lines": [
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-0",
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-1",
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-2",
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-4",
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-3",
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-5",
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-6",
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-7",
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-8",
        "NL-HaNA_3761_0005-page-9-col-0-tr-0-line-9"
    ],
    "text": "W)

## Simpele frequentielijsten

Een eerste stap is kijken wat de frequente woorden zijn in de resoluties van een specifiek inventaris. 

In de resoluties beginnen veel domeinspecifieke inhoudstermen met een hoofdletter. Dat is een goed eerste filter om inzicht te krijgen in de inhoud.

In [72]:
# maak een nieuwe counter aan
term_freq = Counter()

# itereer over all documenten (= paragraphs)
for doc in docs:
    # splits de tekst op alles wat geen alpha-numerical character is
    terms = re.split(r'\W+', doc['text'])
    # een 'list comprehension' om termen te filteren die met een hoofdletter te beginnen
    terms = [term for term in terms if term != '' and term[0].isupper()]
    # update de counter met de lijst van termen in dit document
    term_freq.update(terms)

# print een overzicht van de 100 meest frequente termen
for term, freq in term_freq.most_common(100):
    # print term (links uitgelijnd met een breedte van 30 karakters) en frequentie (rechts uitgelijnd met 6 karakters)
    print(f'{term: <30}{freq: >6}')

Hoogh                           8959
Mogende                         8926
Heeren                          5087
Waer                            4534
Vergaderinge                    3010
Heere                           2978
Gedeputeerden                   2595
Suppliant                       2186
Raedt                           2160
Is                              2157
Requeste                        1928
Ontfangen                       1644
Requefte                        1609
State                           1534
Missive                         1423
Provincie                       1414
Miflive                         1319
Paerden                         1306
Staten                          1241
De                              1164
Op                              1137
Supplianten                     1126
Regiment                        1111
Refolutie                       1065
A                               1047
Resolutie                        935
Copie                            932
G

Termen hoeven geen individuele woorden te zijn, maar kunnen ook woord bi-grammen of groter woord n-grammen zijn.

In [73]:
# maak een nieuwe counter aan
term_freq = Counter()

ngram_size = 3

# itereer over all documenten (= paragraphs)
for doc in docs:
    # splits de tekst op alles wat geen alpha-numerical character is
    terms = re.split(r'\W+', doc['text'])
    # creeër ngrammen door elk woor te verbinden met de n-1 opvolgende woorden
    ngram_terms = [' '.join(terms[ti:ti+ngram_size]) for ti, term in enumerate(terms[:-(ngram_size-1)])]
    # een 'list comprehension' om termen te filteren die met een hoofdletter te beginnen
    terms = [term for term in ngram_terms if term != '' and term[0].isupper()]
    # update de counter met de lijst van termen in dit document
    term_freq.update(terms)

# print een overzicht van de 100 meest frequente termen
for term, freq in term_freq.most_common(100):
    # print term (links uitgelijnd met een breedte van 30 karakters) en frequentie (rechts uitgelijnd met 6 karakters)
    print(f'{term: <30}{freq: >6}')

Waer op gedelibereert           1842
Raedt van State                 1292
Is ter Vergaderinge             1231
Gedeputeerden tot de            1143
Waer op geen                    1038
Hoogh Mogende Gedeputeerden     1013
Hoogh Mogende geliefden          961
Mogende Gedeputeerden tot        920
Vergaderinge rapport te          817
Copie van de                     759
Vergaderinge gelefen de          703
Missive van den                  647
Miflive van den                  570
Copye van de                     570
Ontfangen een Missive            567
Vergaderinge gelesen de          500
Heeren Staten van                492
Gedeputeerden van de             453
Heeren Gedeputeerden van         442
Ontfangen een Miflive            417
Gecommitteerden uyt den          392
Collegie ter Admiraliteyt        388
Paerden op het                   388
Heeren Gecommitteerden uyt       376
Refolutie van den                331
Comptoir op het                  331
Mogende geliefden aen            330
W

## Zoeken met Keywords

Als je interessante woorden/ngrammen hebt gevonden, kun je ze in context zoeken om te kijken wat frequente frasen zijn.

In [74]:
# maak counters voor de tekst voorafgaand en volgend op een keyword
prefix_freq = Counter()
suffix_freq = Counter()

# zomaar een lijstje veel voorkomende woorden/frasen
term = 'Schip'
term = 'Collegie'
term = 'Gecommitteerden'
term = 'Regiment'
term = 'Regenten'
term = 'Pafport'
term = 'Placaat'
term = 'ophef'
term = 'Raedt'
term = 'Hoogh Mogende extraordinaris'
term = 'Collegie ter Admiraliteyt'
term = 'Heeren Gedeputeerden van'
term = 'Generaliteyts Reeckenkamer'
term = 'Compagnie'
term = 'Magistraet' # van
term = 'Ambaffadeur' # (van|aan het hof van|tot)
term = 'Hoogh Mogende Gedeputeerden'

# itereer over de documenten
for doc in docs:
    # zoek de term en bijbehorende context op in het document
    for context in get_keyword_context(doc['text'], term, context_size=5):
        # tel hoe vaak elke prefix/suffix voorkomt
        prefix_freq.update([context['prefix']])
        suffix_freq.update([context['suffix']])
    
# toont een overzicht van de meest frequente prefixes en suffixes
for prefix, freq in prefix_freq.most_common(10):
    print(f'{prefix: <30}{freq: >6}')

print('\n')

for suffix, freq in suffix_freq.most_common(100):
    print(f'{suffix: <30}{freq: >6}')



de Laet, ende andere haer         37
van Effen, ende andere haer       36
van Welderen, ende andere haer    29
van Essen, ende andere haer       22
van Lamsweerde, ende andere haer    22
Heeren Tulleken, ende andere haer    21
Welderen , ende andere haer       16
Essen , ende andere haer          16
Tulleken , ende andere haer       16
van Essen ende andere haer        14


tot de saecken van de             91
tot de faecken van de             57
tot de Militaire saecken ,        32
tot de Militaire saecken, om      21
tot de Militaire faecken ,        21
tot de {aecken van de             20
tot de Militaire faecken ‚        16
tot de Militaire faecken, om      13
tot de Militaire {aecken, om      12
tot de Militaire saecken ‚        11
tot de Buytenlandtsche saecken ,    11
tot de Militaire {aecken ,         9
tot de faecken van het             9
tot de Militaire {aecken ‚         9
tot de Buytenlandtsche saecken, om     8
tot de saecken van het             7
tot de faccken van de    

## Fuzzy zoeken

Omdat er allerlei OCR fouten en spellingsvarianten voorkomen van een keyword/frase, mis je met exact zoeken allerlei voorkomens. De Fuzzy search module biedt mogelijkheden om inexact te zoeken naar tekst strings die lijken op een geven set van keywords/frasen.

In de configuratie kun je aangeven hoeveel een string van een keyword mag afwijken om als match te tellen.

In [75]:
# importeer de fuzzy keyword searcher class en de phrase model class
from republic.fuzzy.fuzzy_keyword_searcher import FuzzyKeywordSearcher
from republic.fuzzy.fuzzy_phrase_model import PhraseModel

# configuratie van de fuzzy searcher
fuzzy_search_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# creeër een nieuwe fuzzy searcher met bovenstaande configuratie 
fuzzy_searcher = FuzzyKeywordSearcher(fuzzy_search_config)


# keywords mag een lijst van strings zijn:
keywords = [
    'Raedt van State',
    'Ambaffadeur aan het hof van',
    'Generaliteyts Reeckenkamer'
]

# of je kunt per keyword extra gegevens toevoegen:
# 'variants': variante voorkomens die je onder hetzelfde keyword wilt tellen
# 'label': een categorizerend label
# 'distractors': keywords die qua tekst-afstand op het keyword lijken maar toch echt iets anders zijn
keywords = [
    {
        'keyword': 'Raedt van State',
    },
    {
        'keyword': 'Hoogh Mogende Gedeputeerden tot de Militaire saecken'
    },
    {
        'keyword': 'Hoogh Mogende Gedeputeerden tot de Buytenlandtsche saecken'
    },
    {
        'keyword': 'Ambaffadeur aan het hof van',
        'variants': [
            'Ambaffadeur van hare'
        ]
    },
    {
        'keyword': 'Generaliteyts Reeckenkamer'
    }
]

# transformeer je lijst met keywords in een phrase model
phrase_model = PhraseModel(model=keywords)
# indexeer de keywords uit het phrase model met de fuzzy searcher
fuzzy_searcher.index_keywords(phrase_model.get_keywords())
# indexeer eventuele varianten van de keywords
fuzzy_searcher.index_spelling_variants(phrase_model.variants)

In [76]:
import json

# itereer over de documenten
for doc in docs:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = fuzzy_searcher.find_candidates(doc['text'], use_word_boundaries=True, include_variants=True)
    # check of er matches zijn en print de tekst en de match objecten
    if len(matches) > 0:
        print(doc['text'])
        for match in matches:
            print(json.dumps(match, indent=4))
        break


Ontfangen cen Mifliye. van den’ Refidendt Bilderbeeck , gefchreven tor Keulen den negen en twintighiten der voorlede macndt December, hpudende, dat behalren het geene hy zedert den vier en twintigh{ten December {eventien hondegdt vier tot den eerften Julii {gventien hondefdt vijf aen arme Paffagters haddeverfehooien, ende oock in fijn voorige Dêclatatic was goedtgedaen , dat haer Hoogh Mogende hadde gelieft hem by der fclver Refolutie van den vierden December laetítleden te authoriferen, om noch tot {eltigh rijcxdaelders of hondert en vijftigh guldens toe aen arme Paflagiers te mogen uytgeven , dát nu bléëeuft de Specikd catie nevens de“op®emclde Refolutie by (jn halfjarige Declaratie gewoaght , dat hy zedert den eerlten Julii voorfchr@%e tt den {es en twintigh ften December 176% defcfhonderd vijfieh ed nensaen arme Paflig<isioch hadde verfchooren vier en feftislrgulf®hs acht ftuyvers, de welcke hy mede in de-opgemelde Declaratie inbraghte, op hoope , dat.haer Hoogh Mogende {ouden geli

In [77]:
# creeër counters voor de varianten van elk keyword
variants = defaultdict(Counter)

# itereer over de documenten
for doc in docs:
    # gebruik 'find_candidates' om kandidaat matches te vinden in de tekst
    matches = fuzzy_searcher.find_candidates(doc['text'], use_word_boundaries=True, include_variants=True)
    # voor elke match, tel de matchende string als variant van het matchende keyword
    for match in matches:
        variants[match['match_keyword']].update([match['match_string']])

# toon voor elk van de gevonden keywords een overzicht van de meest frequente varianten
for keyword in variants:
    print('Keyword:', keyword)
    print('------------------------------------')
    for variant, freq in variants[keyword].most_common():
        print(f'{variant: <20}\t{freq: >6}')
    print('\n')


Keyword: Raedt van State
------------------------------------
Raedt van State     	  1251
Raedt van Staten    	   163
Raede van State     	    40
Racdt van State     	    25
Raed van State      	    19
Raedt van Stare     	    11
Raedr van State     	     8
Raedt van Státe     	     8
Raede van Staten    	     7
Raedt van Sate      	     6
Raedt van. State    	     6
Raedt van’ State    	     6
Rade van State      	     5
Raedt van $tate     	     5
Raedt van Srate     	     5
Raedt. van State    	     5
Raedt ran State     	     4
Raedrvan State      	     3
Raedt van Stite     	     3
Raedt ‘van State    	     3
Raédt van State     	     3
Ráedt van State     	     3
Raede van Stare     	     3
Raeden van State    	     3
Raedt van Stater    	     2
Rúáedt van State    	     2
Raedt’van State     	     2
Rede van State      	     2
Raédt van Staten    	     2
Redt van State      	     2
Raedt van'State     	     2
Raedt’ van State    	     2
Racdr van State     	     2
Raedt van Stas